**Lab 4: FIR filters and envelopes**

The goal of this lab is to learn how to implement FIR filters in Python, and how we can use them to improve our synthesis.

In [ ]:
import os
import numpy as np
import librosa
import IPython.display as ipd
from scipy import signal
import matplotlib.pyplot as plt

from util import load_audio, plot_signals, plot_spectrogram, plot_mean_spectrogram

In [ ]:
import numpy as np

def averaging_filter(x, N):
    h = np.ones(N) / N
    y = np.convolve(x, h, mode='same')
    return y


**Envelope Detector**

In [ ]:
import numpy as np

def envelope(x, N): 
    rectified = np.abs(x)
    envelope = averaging_filter(rectified, N)
    return envelope


In [ ]:
filepath = "./sis1_group103/e1pluck.wav"
ref, fs = load_audio(filepath)

enveloped_ref = envelope(ref, 900)

scale = 1.4 # had to introduce this bc enveloepe was too small compared to ref
# i had to make the main clean part of the signal match with the envelope
enveloped_ref_scaled = enveloped_ref * scale

In [ ]:
rectified = np.abs(ref)
plot_signals([ref, rectified, enveloped_ref_scaled], fs, t_start = 0, t_end= 3, name=['Reference signal', 'Abs(Ref)', 'Envelope'], mode='lines')


2.5 Now let's apply this envelope to the synthesized signal. Copy the code from Lab3 Ex. 2.1 and generate the synthesized signal. Then multiply the synthesized signal by the envelope. Note that both signal and envelope should have the same length. You can define a time vector of the same length of the envelope to create the signal:

In [ ]:
weights = [1, 0.5234, 0.3644, 0.0449, 0.0290, 0.3463, 0.1484, 0.1222, 0.0299, 0.0362]
def synthesize(f0, phi, Ak, t):
  y = 0
  for k in range(1, len(Ak) + 1):
    y += Ak[k-1] * np.cos(2*np.pi*k*f0*t + k*phi - (k-1)*np.pi/2)
  return y

N_samples = len(enveloped_ref_scaled)
t = np.arange(N_samples) / fs
synt = synthesize(329.10, 4.75, weights, t=t)

enveloped_synt = enveloped_ref_scaled * synt
plot_signals([ref, enveloped_synt], sr=fs, t_start=0, t_end=3, name = ['reference', 'enveloped synt']) 


2.6. Compare the spectrograms of the reference signal and the synthesized signal. What are the main differences?

In [ ]:
from util import load_audio, plot_signals, plot_spectrogram, plot_mean_spectrogram, plot_spectrum_at
from scipy.signal import spectrogram

ff_synt, tt_synt, S_synt = spectrogram(enveloped_synt, fs=fs, nperseg=1024, noverlap=512)
ff, tt, S_ref = spectrogram(ref, fs=fs, nperseg=1024, noverlap=512)

plot_spectrogram(ff_synt, tt_synt, S_synt)
plt.xlim(0, 3)        # limit time axis 
plt.ylim(0, 4000)     # limit frequency axis
plt.title("Spectrogram of synthesis")
plt.show()

plot_spectrogram(ff, tt, S_ref)
plt.xlim(0, 3)        # limit time axis 
plt.ylim(0, 4000)     # limit frequency axis
plt.title("Spectrogram of reference sound")
plt.show()

2.7 Listen to the synthesized signal and compare it to the reference.



In [ ]:
from IPython.display import Audio
print("Synthesis  Sound")
Audio(enveloped_synt, rate=fs)

In [ ]:
print("Reference Sound")
Audio(ref, rate=fs)

Im actually pretty happy with how this turned out they sound pretty similar and the envelope made a huge difference in how similar the pluck sounds. Theyre identical in volume now which helps and the only (inevitable) difference is that the reference sounds more organic. 